In [3]:
from neo4j import GraphDatabase
import pandas as pd
import pprint

In [4]:
graphdb = GraphDatabase.driver(uri = 'bolt://localhost:7687', auth=('neo4j', 'haslo'))

In [5]:
session = graphdb.session()
session

In [8]:
path = r'C:\projects\Neo4j'

In [9]:
df = pd.read_csv(path + r'\Airplane_Crashes_and_Fatalities_Since_1908_20190820105639.csv')
df

,Date,Time,Location,Operator,Flight #,Route,AC Type,Registration,cn/ln,Aboard,Aboard Passangers,Aboard Crew,Fatalities,Fatalities Passangers,Fatalities Crew,Ground,Summary
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,1.0,1.0,1.0,0.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,09/07/1909,NaN,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,NaN,1.0,0.0,1.0,1.0,0.0,0.0,0.0,Eugene Lefebvre was the first pilot to ever be...
2,07/12/1912,06:30,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,0.0,5.0,5.0,0.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
3,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,NaN,NaN,Curtiss seaplane,NaN,NaN,1.0,0.0,1.0,1.0,0.0,1.0,0.0,The first fatal airplane accident in Canada oc...
4,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,NaN,NaN,14.0,NaN,NaN,0.0,The airship flew into a thunderstorm and encou...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4962,04/16/2019,11:00,"Puerto Montt, Chile",Archipelagos Service Aereos,NaN,Puerto Montt - Ayacara,Pilatus-Britten Norman BN-2B-27 Islander,CC-CYR,2169,6.0,5.0,1.0,6.0,5.0,1.0,0.0,"While the aircraft was in the initial climb, p..."
4963,05/05/2019,18:30,"Near Monclava, Mexico",TVPX Aircraft Solutions,NaN,Las Vegas - Monterrey,Canadair CL-600-2B16-Challenger,N601VH,5043,13.0,11.0,2.0,13.0,11.0,2.0,0.0,The aircraft crashed while en route on a retur...
4964,05/05/2019,18:30,"Moscow, Russia",Aeroflot Russian International Airlines,NaN,Moscow - Murmansk,Sukhoi Superjet-100-95B,RA-89098,95135,78.0,73.0,5.0,41.0,40.0,1.0,0.0,Forty-five minutes after taking off from Mosco...
4965,06/03/2019,13:00,"Near Lipo, India",Military - Indian Air Force,NaN,Jorhat-Rowriah - Mechuka,Antonov An-32,K2752,1009,13.0,5.0,8.0,13.0,5.0,8.0,0.0,Crashed about 34km WNW of Mechuka.


In [10]:
#replacing NaN with 0
df.fillna(0,inplace=True)

In [12]:
#removal "" from Summary
for i, o in enumerate(df['Summary']):
    if type(o) == str and '"' in o:
        df['Summary'][i] = df['Summary'][i].replace('"','')

In [16]:
#adding data
list_op = []
for i in range(len(df['Date'])):
    q1 = 'create (s:airplane{registration:"' +str(df['Registration'][i]) + '", type:"' +str(df['AC Type'][i])  +
        '", nr_structural:"' +str(df['cn/ln'][i]) +'", operator:"'+ str(df['Operator'][i]) +'", id:' + str(i) + '})'
    session.run(q1)
    
    q2 = 'create (w:accident{place:"' +str(df['Location'][i]) +'", time:"'+ str(df['Time'][i]) + 
        '", date:"' + str(df['Date'][i]) + '", summary:"' +str(df['Summary'][i]) +'", id:' + str(i) + '})'
    session.run(q2)
    
    q3 = 'create (l:flight{nr_flight:"' +str(df['Flight #'][i]) +'", route:"'+ str(df['Route'][i]) +'", id:' + str(i) + '})'
    session.run(q3)
    
    q4 = 'create (n:aboard{passangers:' +str(df['Aboard Passangers'][i]) +', crew:'+ 
        str(df['Aboard Crew'][i]) + ', id:' + str(i) + '})'
    session.run(q4)

    q5 = 'create (z:died{fatalities_passangers:' +str(df['Fatalities Passangers'][i]) + ', fatalities_crew:' + 
        str(df['Fatalities Crew'][i]) + ', fatalities_ground:' +str(df['Ground'][i])+', id:' + str(i) + '})'
    session.run(q5)
    
    #adding unique operator names
    if df['Operator'][i] not in list_op:
        list_op.append(df['Operator'][i])
        q6 = 'create (o:operator{name:"' +str(df['Operator'][i]) +'", id:"' + str(len(list_op)) + '"})'
        session.run(q6)

In [18]:
#connecting
q7 = 'MATCH (w:accident) MATCH (s:airplane) WHERE w.id = s.id MERGE (s)-[:HAD_ACCIDENT]->(w)'
session.run(q7) 
q8 = 'MATCH (l:flight) MATCH (s:airplane) WHERE l.id = s.id MERGE (s)-[:MADE_FLIGHT]->(l)'
session.run(q8)
q9 = 'MATCH (l:flight) MATCH (n:aboard) WHERE l.id = n.id MERGE (n)-[:WERE_FLYING]->(l)'
session.run(q9)
q10 = 'MATCH (l:flight) MATCH (z:died) WHERE l.id = z.id MERGE (z)-[:VICTIMS]->(l)'
session.run(q10)
q11 = 'MATCH (o:operator) MATCH (s:airplane) WHERE o.name = s.operator MERGE (s)-[:BELONGS_TO]->(o)'
session.run(q11)

In [13]:
#adding new data
df_new = pd.read_csv(path + r'\Last_worlds_2022_11_13.csv')
df_new

,Date,Airlane,Flight_number,Last_words
0,07 Jul 1962,Alitalia,771,"Unable to make out your last message, will you..."
1,07 May 1964,Pacific Air Lines,773,Skipper's shot! We've been shot. I was trying...
2,08 Nov 1965,American Airlines,383,Have you still got the runway OK? Ah .. just b...
3,30 Jun 1967,Thai International,601,I have no radar contact with you.
4,06 Nov 1967,Trans World Airlines,159,Not very # far off the runway. Sure as # isn't.
...,...,...,...,...
81,01 Jun 2009,Air France,447,"Damn it, we're going to crash... This can't be..."
82,17 JuL 2007,Tam,3054,"Oh my god, oh my god"
83,15 Jan 2009,US Airways,1549,We're gonna be in the Hudson.
84,12 Feb 2009,Continental Express,3407,We're down.


In [14]:
dict_month = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

In [15]:
data = []
for i, d in enumerate(df_new['Date']):
    dz = d.replace(' ', '')[:2]
    m = d.replace(' ', '')[2:5]
    r = d.replace(' ', '')[-4:]
    if i == 19:   #exception, handled non-automatically
        m = 'Nov'
    m = m[0] + m[1:].lower()   #sometimes the second letter was capitalized
    data.append(dict_month[m]+'/'+dz+'/'+r)

In [16]:
df_new['Date'] = data
df_new

,Date,Airlane,Flight_number,Last_words
0,07/07/1962,Alitalia,771,"Unable to make out your last message, will you..."
1,05/07/1964,Pacific Air Lines,773,Skipper's shot! We've been shot. I was trying...
2,11/08/1965,American Airlines,383,Have you still got the runway OK? Ah .. just b...
3,06/30/1967,Thai International,601,I have no radar contact with you.
4,11/06/1967,Trans World Airlines,159,Not very # far off the runway. Sure as # isn't.
...,...,...,...,...
81,06/01/2009,Air France,447,"Damn it, we're going to crash... This can't be..."
82,07/17/2007,Tam,3054,"Oh my god, oh my god"
83,01/15/2009,US Airways,1549,We're gonna be in the Hudson.
84,02/12/2009,Continental Express,3407,We're down.


In [32]:
#sentence node
for i in range(len(df_new)):
    q12 = 'create (z:sentence{date:"' +str(df_new['Date'][i]) + '", operator:"' + str(df_new['Airlane'][i]) + 
    '", nr_flight:"' +str(df_new['Flight_number'][i])+ '", words:"' +  str(df_new['Last_words'][i])+ '"})'
    session.run(q12)

In [33]:
#connecting words - accident
q13 = 'MATCH (z:words) MATCH (w:accident) WHERE z.date = w.date MERGE (z)-[:WORDS_SPOKEN]->(w)'
session.run(q13) 

In [31]:
#query showing on which route the most people died in total
q14 = 'MATCH (l:flight) with l match (l) <- [:VICTIMS] - (z:died) return l.route as route, \
    z.fatalities_passangers + z.fatalities_crew + z.fatalities_ground as sum order by sum DESC'
nodes = session.run(q14)
for node in nodes:
    print(node['route'], int(node['sum']))

Boston - Los Angeles 2842
Boston - Los Angeles 2815
Tenerife - Las Palmas / Tenerife - Las Palmas 583
Tokyo - Osaka 520
New Delhi - Dhahran / Chimkent - New Delhi 349
Paris - London 346
Montreal - London 329
Riyadh - Jeddah 301
Amsterdam - Kuala Lumpur 298
Bandar Abbas - Dubai 290


In [30]:
#the total number of deaths from flights that included Warsaw
q14a = "MATCH (l:flight) where l.route =~ '.*Warsaw.*' with l match (l) <- [:VICTIMS] - (z:died) return l.route as route, \
    z.fatalities_passangers + z.fatalities_crew + z.fatalities_ground as sum order by sum DESC"
nodes = session.run(q14a)
for node in nodes:
    print(node['route'], int(node['sum']))

Warsaw - New York City 183
Warsaw, Poland - Smolensk Russia 96
New York City - Warsaw 87
Warsaw - Cracow 53
Berlin - Warsaw 33
Warsaw - Moscow 31
Warsaw - Miroslawiec 20
Warsaw, Poland - Bucharest, Romania 14
Warsaw - Moscow 9
Warsaw - Prague - Strasbourg - Paris 5
Krakow - Warsaw 4
Stockholm-Copenhagen-Warsaw 0


In [19]:
#number of accidents in a given year
q15 = 'match (w:accident) return right(w.date,4) as year, count(right(w.date,4)) as count_accident order by count_accident desc'
nodes = session.run(q15)
for node in nodes[:10]:
    print('year:', node['year'], 'count_accident:', node['count_accident'])

year: 1966 count_accident: 308
year: 1967 count_accident: 308
year: 1965 count_accident: 300
year: 1964 count_accident: 283
year: 1969 count_accident: 276
year: 1979 count_accident: 262
year: 1968 count_accident: 260
year: 1978 count_accident: 255
year: 1970 count_accident: 237
year: 1981 count_accident: 212


In [21]:
#a given type of airplane belonging to the operator
q16 = "MATCH (o:operator) with o match (o) <- [:BELONGS_TO] - (s:airplane {type:'Douglas DC-3'}) return o, s"
nodes = session.run(q16)
for node in nodes[:10]:
    print('operator:', node['o']['name'], 'airplane_type:', node['s']['type'])

operator: Sadai airplane_type: Douglas DC-3
operator: Finnair airplane_type: Douglas DC-3
operator: Aerolineas Litoral Argentina airplane_type: Douglas DC-3
operator: Lloyd Aéreo Boliviano airplane_type: Douglas DC-3
operator: Philippine Air Lines airplane_type: Douglas DC-3
operator: Snow Valley Ski Lines airplane_type: Douglas DC-3
operator: Taxi Aereo de airplane_type airplane_type: Douglas DC-3
operator: Aden Airways airplane_type: Douglas DC-3
operator: TASSA airplane_type: Douglas DC-3
operator: Aerolineas Abaroa airplane_type: Douglas DC-3


In [23]:
#how many airplane of a given type belong to the operator
q16a = "MATCH (o:operator) with o match (o) <- [:BELONGS_TO] - (s:airplane {type:'Douglas DC-3'}) return o.name, \
    count(s) as count_airplane order by count_airplane desc"
nodes = session.run(q16a)
for node in nodes[:10]:
    print('operator:', node['o.name'], 'count_airplane:', node['count_airplane'])

operator: Philippine Air Lines count_airplane: 10
operator: Indian Airlines count_airplane: 9
operator: Ethiopian Airlines count_airplane: 8
operator: Cruzeiro count_airplane: 7
operator: VASP count_airplane: 6
operator: AVIANCA count_airplane: 5
operator: KLM Royal Dutch Airlines count_airplane: 5
operator: American Airlines count_airplane: 5
operator: Aigle Azur count_airplane: 5
operator: Garuda Indonesia Airlines count_airplanecount_airplane: 5


In [27]:
#miejsca wypadkow dla dwoch roznych operatorow
q17 = "MATCH (w:accident) with w match (w) <- [:WORDS_SPOKEN] - (z:sentence) where z.operator = 'United Airlines' \
    return z.operator, w.place UNION MATCH (w:accident) with w match (w) <- [:WORDS_SPOKEN] - (z:sentence) \
    where z.operator = 'Western Airlines' return z.operator, w.place" 
nodes = session.run(q17)
for node in nodes:
    print('operator:', node['z.operator'], 'place:', node['place'])

operator: United Airlines place: Midway Airport, Chicago, Illinois
operator: United Airlines place: Near Rawalpindi, Pakistan
operator: United Airlines place: Portland, Oregon
operator: United Airlines place: Honolulu, Hawaii
operator: United Airlines place: Talourow Island, USSR
operator: United Airlines place: Sioux City, Iowa
operator: United Airlines place: Glen Burnie, Maryland
operator: United Airlines place: Near Colorado Springs, Colorado
operator: United Airlines place: New York City, New York
operator: United Airlines place: Arlington, Virginia.
operator: United Airlines place: Shanksville, Pennsylvania
operator: Western Airlines place: Mexico City, Mexico


In [29]:
q18 = 'match (z:died) - [:VICTIMS] -> (l:flight) <- [:WERE_FLYING] - (p:aboard) where \
    z.fatalities_passangers + z.fatalities_crew < p.passangers + p.crew and p.passangers + p.crew > 0 \
    and z.fatalities_passangers + z.fatalities_crew > 0 return l, p, z'
nodes = session.run(q18)
for node in nodes[:10]:
    print(node['l']['route'],  node['p']['passangers'] + node['p']['crew'] - node['z']['fatalities_passangers'] - node['z']['fatalities_crew'])

Turku - Mariehamn 2.0
Rosario - Buenos Aires 2.0
Cotabato - Malabang - Iligan - Cayagan de Oro 1.0
Kagoshima - Oita 22.0
Buenos Aires - Lima 3.0
Travis - Hickam - Clark 5.0
Palma de Mallorca - Ibiza 27.0
Dublin, Ireland - Addis Ababa 6.0
Stockholm - Hultsfred - Halmstad - Ängelholm 12.0
Rome - Athens 23.0


In [276]:
q18a = 'match (z:died) - [:VICTIMS] -> (l:flight) <- [:WERE_FLYING] - (p:aboard) where \
    z.fatalities_crew + z.fatalities_passangers < p.passangers + p.crew and p.passangers + p.crew > 0 \
    and z.fatalities_passangers + z.fatalities_crew > 0 return count(l) as how_many'
nodes = session.run(q18a)
for node in nodes:
    print(node['how_many'])

1639


In [279]:
#before index
q19 = "profile MATCH (o:operator) with o match (o) <- [:BELONGS_TO] - (s:airplane {type:'Douglas DC-3'}) return o, s"
session.run(q19)

In [307]:
#add index
q20 = 'create index ind1 for (s:airplane) on (s.type)'
session.run(q20)

In [291]:
#after index
q21 = "profile MATCH (o:operator) with o match (o) <- [:BELONGS_TO] - (s:airplane {type:'Douglas DC-3'}) return o, s"
session.run(q21)

In [292]:
#before index
q22 = "profile MATCH (w:accident) with w match (w) <- [:WORDS_SPOKEN] - (z:sentence) where \
    z.operator = 'United Airlines' return z.operator, w.place UNION MATCH (w:accident) \
    with w match (w) <- [:WORDS_SPOKEN] - (z:sentence) where z.operator = 'Western Airlines' return z.operator, w.place"
session.run(q22)

In [304]:
#add index
q23 = 'create index ind2 for (z:sentence) on (z.operator)'
session.run(q23)

In [295]:
#after index
q22 = "profile MATCH (w:accident) with w match (w) <- [:WORDS_SPOKEN] - (z:sentence) where \
    z.operator = 'United Airlines' return z.operator, w.place UNION MATCH (w:accident) \
    with w match (w) <- [:WORDS_SPOKEN] - (z:sentence) where z.operator = 'Western Airlines' return z.operator, w.place"
session.run(q22)

In [308]:
#shows all superimposed indexes
q23 = 'show indexes'
nodes = session.run(q23)
for node in nodes:
    print(node)

<Record id=2 name='ind1' state='ONLINE' populationPercent=100.0 uniqueness='NONUNIQUE' type='BTREE' entityType='NODE' labelsOrTypes=['samolot'] properties=['typ'] indexProvider='native-btree-1.0'>
<Record id=1 name='ind2' state='ONLINE' populationPercent=100.0 uniqueness='NONUNIQUE' type='BTREE' entityType='NODE' labelsOrTypes=['zdanie'] properties=['operator'] indexProvider='native-btree-1.0'>


In [312]:
#procedure
q24 = "call apoc.custom.asProcedure('passangers_sum','MATCH (l:lot {route: $atr1}) <- [:WERE_FLYING] - (p:aboard) \
    return $atr1, p.crew + p.passangers as passangers_sum', 'read', [['passangers_sum', 'float']],[['atr1', 'string']])"
session.run(q24)

In [313]:
q25 = 'call custom.passangers_sum("Stockholm-Copenhagen-Warsaw")'
nodes = session.run(q25)
for node in nodes:
    print(node['passangers_sum'])

129.0


In [314]:
q26 = 'call custom.passangers_sum("Boston - Los Angeles")'
nodes = session.run(q26)
for node in nodes:
    print(node['passangers_sum'])

92.0
65.0
